In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
from pandas import DataFrame
import pandas_gbq
import io

ModuleNotFoundError: No module named 'pandas_gbq'

In [ ]:
conda install pandas-gbq --channel conda-forge

In [4]:
%load_ext google.cloud.bigquery

In [7]:
#Create an environment variable to hold our google application credentials that are stored in the .json file
#given by GBQ when we first made the database
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/Users/jackc/OneDrive/Documents/MSBA/Applied Data Analytics/Wedge/cloughwedge2021-3249989ff592.json"

In [8]:
#Variables that contain necessary credentials to connect to our GBQ instance
client = bigquery.Client(project = 'cloughwedge2021')
gbq_proj_id = "cloughwedge2021"

In [9]:
#Iterate through our clean wedge files folder, put all of the names to a list
path = os.getcwd()
files = []

for file_name in os.listdir(path + '\\Wedge_clean_files\\'):
    files.append(file_name)
    

In [5]:
#The import.

#Provide the project id and the database's name

projID = "cloughwedge2021."
database = "transactions"

#This will loop through each of the 50+ files in the wedge file folder and import each file's contents.

#Using the Pandas package and its read_csv function, we create a data frame and fill it with the values in each csv file.
#The data frame has the same variable names as they are listed in the csv files. If I were to do this part any different,
#I would probably use lowercase letters in the column names. It was a massive pain in the ass to go through and make sure
#that each time I used a variable or several variables in the SQL statements, the variable matched where I originally
#claimed it. Obviously, I would miss a few then have to go back and see which ones were not capitalized like they should've
#been. ANYWAYS...


for file in files:
    TableName = projID + file[:-4]
    df = pd.read_csv(path + '\\Wedge_clean_files\\' + file, names =['DateTime','Register_Num','Emp_Num',"Trans_Num", "UPC","Description",'Trans_Type',"Trans_SubType",'Trans_Status',
                                    'Department','Quantity','Scale','Cost','Unit_Price','Total','Reg_Price','Alt_Price','Tax',
                                    "TaxExempt",'Foodstamp','WicAble','Discount','memDiscount','Discountable','DiscountType',
                                    'Voided','PercentDiscount','ItemQtty','volDiscType','Volume','VolSpecial','mixMatch','Matched'
                                    ,'MemType','Staff','NumFlag','itemStatus','TenderStatus','CharFlag','varflag','BatchHeaderID','Local',
                                    'Organic','Display','Receipt','Card_No','Store','Branch','Match_ID','Trans_ID'])
    
    #Here's what solves an error I and a classmate were having, where the import was giving a warning message about
    #the types of varaibles that were being imported. This is solved by converting the variables in question to
    #boolean type, which seems to have solved the issue.
    
    cols_to_bool = ["MemType", "Staff", "BatchHeaderID", "Display"]
    df[cols_to_bool] = df[cols_to_bool].astype('bool')

    #Here's where the datagrame gets sent to our GBQ instance, and here's also where that capitalization
    #"pain in the ass" thing comes in. Essentially, we're gonna assign a schema to the dataframe, enlarge the chunk
    #size to 500000 so that it works faster and GBQ doesn't get overrun with how many requests were making to it,
    #and ensuring that the import will override a table if it already exists (RIP after having to do this
    #multiple times because it took FOREVER. Also, the table schema calls requires asssigning variable types to
    #each column. This needs to be perfectly in line with where we gave each variable a type when we created
    #the data frame.
    
    DataFrame.to_gbq(df,TableName,gbq_proj_id,chunksize = 500000, if_exists = "replace",
                    table_schema = [{'name': 'DateTime', 'type': 'Timestamp'}, {'name': 'Register_Num', 'type': 'Float'},
{'name': 'Emp_Num', 'type': 'Float'}, {'name': 'Trans_Num', 'type': 'Float'},
{'name': 'UPC', 'type': 'string'}, {'name': 'Description', 'type': 'string'},
{'name': 'Trans_Type', 'type': 'string'}, {'name': 'Trans_SubType', 'type': 'string'},
{'name': 'Trans_Status', 'type': 'string'}, {'name': 'Department', 'type': 'Float'},
{'name': 'Quantity', 'type': 'float'}, {'name': 'Scale', 'type': 'Float'},
{'name': 'Cost', 'type': 'float'}, {'name': 'Unit_Price', 'type': 'float'},
{'name': 'Total', 'type': 'float'}, {'name': 'Reg_Price', 'type': 'float'},
{'name': 'Alt_Price', 'type': 'float'}, {'name': 'Tax', 'type': 'float'},
{'name': 'TaxExempt', 'type': 'float'}, {'name': 'Foodstamp', 'type': 'Float'},
{'name': 'WicAble', 'type': 'Float'}, {'name': 'Discount', 'type': 'float'},
{'name': 'memDiscount', 'type': 'float'}, {'name': 'Discountable', 'type': 'float'},
{'name': 'DiscountType', 'type': 'float'}, {'name': 'Voided', 'type': 'float'},
{'name': 'PercentDiscount', 'type': 'float'}, {'name': 'ItemQtty', 'type': 'float'},
{'name': 'volDiscType', 'type': 'float'}, {'name': 'Volume', 'type': 'float'},
{'name': 'VolSpecial', 'type': 'float'}, {'name': 'mixMatch', 'type': 'float'},
{'name': 'Matched', 'type': 'float'}, {'name': 'MemType', 'type': 'Boolean'},
{'name': 'Staff', 'type': 'Boolean'}, {'name': 'NumFlag', 'type': 'float'},
{'name': 'itemStatus', 'type': 'float'}, {'name': 'TenderStatus', 'type': 'float'},
{'name': 'CharFlag', 'type': 'string'}, {'name': 'varFlag', 'type': 'float'},
{'name': 'BatchHeaderID', 'type': 'boolean'}, {'name': 'Local', 'type': 'float'},
{'name': 'Organic', 'type': 'float'}, {'name': 'Display', 'type': 'Boolean'},
{'name': 'Receipt', 'type': 'float'}, {'name': 'Card_No', 'type': 'float'},
{'name': 'Store', 'type': 'float'}, {'name': 'Branch', 'type': 'float'},
{'name': 'Match_ID', 'type': 'float'}, {'name': 'Trans_ID', 'type': 'float'}])

6it [12:43, 127.22s/it]
7it [13:38, 116.89s/it]
6it [13:15, 132.56s/it]
c:\users\jackc\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (43) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
6it [13:07, 131.31s/it]
6it [14:32, 145.43s/it]
3it [04:54, 98.11s/it]
3it [04:50, 96.76s/it]
2it [04:39, 139.84s/it]
7it [14:15, 122.25s/it]
7it [13:59, 119.86s/it]
6it [13:44, 137.37s/it]
1it [01:05, 65.82s/it]
c:\users\jackc\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
7it [12:55, 110.73s/it]
1it [02:39, 159.86s/it]
6it [13:23, 133.95s/it]
1it [00:50, 50.50s/it]
6it [13:36, 136.16s/it]
1it [00:41, 41.51s/it]
6it [12:59, 129.89s

In [11]:
#Here I did a count of how many records we have in the GBQ isntance to make sure we got everything in, which it seems like
#we did! Also, I wanted to find a different way of using the SQL commands in python other than having them in comment
#format and assigning that to a variable to be called later, so I foud this. I didn't use it later on, but it still
#looks nicer.

%%bigquery
SELECT
    COUNT(UPC) 
FROM `cloughwedge2021.cloughwedge2021.transArchive*`

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.41s/rows]


,f0_
0,85760139
